In [1]:
import pandas as pd
import numpy as np
from dash_website import DIMENSIONS, MAIN_CATEGORIES_TO_CATEGORIES

list_indexes = []
for dimension_1 in DIMENSIONS:
    for dimension_2 in DIMENSIONS:
        for category in MAIN_CATEGORIES_TO_CATEGORIES["All"] + ["Phenotypic", "Genetics"] + [f"All_{main_category}" for main_category in MAIN_CATEGORIES_TO_CATEGORIES.keys()]:
            list_indexes.append([dimension_1, dimension_2, category])
indexes = pd.MultiIndex.from_tuples(list_indexes, names=["dimension_1", "dimension_2", "category"])

list_columns = []
for method in ["all", "union", "intersection"]:
    for correlation_type in ["pearson", "spearman", "number_variables"]:
        list_columns.append([method, correlation_type])
columns = pd.MultiIndex.from_tuples(list_columns, names=["method", "correlation_type"])
correlations = pd.DataFrame(None, index=indexes, columns=columns)

phenotypic = pd.read_feather("../../data/xwas/univariate_correlations/phenotypic.feather").set_index(["dimension_1", "dimension_2"])
genetics = pd.read_feather("../../data/xwas/univariate_correlations/genetics.feather").set_index(["dimension_1", "dimension_2"])

In [2]:
dimension_1, dimension_2 = "Abdomen", "EyesAll"

In [3]:
def load_correlation(key_in_aws):
    correlation_dimension = pd.read_feather(key_in_aws)
    correlation_dimension.drop(index=correlation_dimension.index[correlation_dimension["sample_size"] < 10], inplace=True)

    return correlation_dimension.set_index(["category", "variable"])

In [4]:
correlation_dimension_1 = load_correlation(f"../../data/xwas/univariate_results/linear_correlations_{dimension_1}.feather")
correlation_dimension_2 = load_correlation(f"../../data/xwas/univariate_results/linear_correlations_{dimension_2}.feather")

In [5]:
for category in MAIN_CATEGORIES_TO_CATEGORIES["All"] + [f"All_{main_category}" for main_category in MAIN_CATEGORIES_TO_CATEGORIES.keys()]:
    if "All" in category:
        categories = MAIN_CATEGORIES_TO_CATEGORIES[category.split("_")[1]]
    else:
        categories = [category]

    correlations_1 = correlation_dimension_1.loc[categories]
    correlations_2 = correlation_dimension_2.loc[categories]

    indexes = {}

    indexes["all"] = correlations_1.index.intersection(correlations_2.index)

    indexes["union"] = correlations_1.index[
        correlations_1["p_value"] < 0.05 / correlations_1.shape[0]
    ].union(correlations_2.index[correlations_2["p_value"] < 0.05 / correlations_2.shape[0]])
    if (~indexes["union"].isin(correlations_1.index)).sum() > 0:
        correlations_to_add = pd.DataFrame(None, index=indexes["union"][~indexes["union"].isin(correlations_1.index)], columns=["p_value", "correlation", "sample_size"])
        correlations_to_add["correlation"] = 0

        correlations_1 = correlations_1.append(correlations_to_add)
    if (~indexes["union"].isin(correlations_2.index)).sum() > 0:
        correlations_to_add = pd.DataFrame(None, index=indexes["union"][~indexes["union"].isin(correlations_2.index)], columns=["p_value", "correlation", "sample_size"])
        correlations_to_add["correlation"] = 0

        correlations_2 = correlations_2.append(correlations_to_add)

    indexes["intersection"] = correlations_1.index[correlations_1["p_value"] < 0.05 / correlations_1.shape[0]].intersection(correlations_2.index[correlations_2["p_value"] < 0.05 / correlations_2.shape[0]])

    for method in ["all", "union", "intersection"]:
        for correlation_type in ["pearson", "spearman"]:
            if len(indexes[method]) <= 1:
                correlations.loc[(dimension_1, dimension_2, category), (method, correlation_type)] = 0
            else:
                correlations.loc[(dimension_1, dimension_2, category), (method, correlation_type)] = correlations_1.loc[indexes[method], "correlation"].corr(correlations_2.loc[indexes[method], "correlation"], method=correlation_type)
            correlations.loc[(dimension_1, dimension_2, category), (method, "number_variables")] = len(indexes[method])

for method in ["all", "union", "intersection"]:
    for correlation_type in ["pearson", "spearman"]:
        correlations.loc[(dimension_1, dimension_2, "Phenotypic"), (method, correlation_type)] = phenotypic.loc[(dimension_1, dimension_2), "correlation"]
        correlations.loc[(dimension_1, dimension_2, "Genetics"), (method, correlation_type)] = genetics.loc[(dimension_1, dimension_2), "correlation"]

c:\Dossier\HMS\Dash-Website\env_website\lib\site-packages\numpy\lib\function_base.py:2634: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Dossier\HMS\Dash-Website\env_website\lib\site-packages\numpy\lib\function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Dossier\HMS\Dash-Website\env_website\lib\site-packages\numpy\lib\function_base.py:2634: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Dossier\HMS\Dash-Website\env_website\lib\site-packages\numpy\lib\function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [6]:
correlations.loc[(dimension_1, dimension_2)]

method                       all                                union  \
correlation_type         pearson  spearman number_variables   pearson   
category                                                                
Alcohol                 0.245172  0.201478               29 -0.200475   
AnthropometryBodySize  -0.498217 -0.619048                8 -0.498217   
AnthropometryImpedance -0.585731      -0.7                5 -0.585731   
ArterialStiffness       0.612142   0.47619                8  0.362174   
BloodBiochemistry       0.077582  0.015873               28  0.053816   
...                          ...       ...              ...       ...   
All_Biomarkers          0.444471  0.661951             3115  0.420716   
All_Phenotypes          0.110952  0.340091              182  0.250916   
All_Diseases            0.362978  0.277877             2073 -0.029624   
All_Environmental       0.378952  0.481816              263  0.433484   
All_Socioeconomics      0.642517  0.749437               88  0.819856   

method                                            intersection            \
correlation_type        spearman number_variables      pearson  spearman   
category                                                                   
Alcohol                -0.244755               12    -0.771148      -1.0   
AnthropometryBodySize  -0.619048                8    -0.779553      -0.8   
AnthropometryImpedance      -0.7                5    -0.585731      -0.7   
ArterialStiffness       0.085714                6         -1.0      -1.0   
BloodBiochemistry      -0.045614               19     0.015564  0.033333   
...                          ...              ...          ...       ...   
All_Biomarkers          0.644366              679     0.633293  0.250494   
All_Phenotypes          0.312096               80     0.744237  0.433162   
All_Diseases           -0.325477               62     0.863583  0.757576   
All_Environmental       0.538802              114     0.670108     0.595   
All_Socioeconomics      0.874603               38     0.897566  0.690909   

method                                   
correlation_type       number_variables  
category                                 
Alcohol                               4  
AnthropometryBodySize                 4  
AnthropometryImpedance                5  
ArterialStiffness                     2  
BloodBiochemistry                     9  
...                                 ...  
All_Biomarkers                       73  
All_Phenotypes                       26  
All_Diseases                         10  
All_Environmental                    49  
All_Socioeconomics                   11  

[92 rows x 9 columns]

In [29]:
correlations.columns = map(str, correlations.columns.tolist())
d = correlations.reset_index()
d

,dimension_1,dimension_2,category,"('all', 'pearson')","('all', 'spearman')","('all', 'number_variables')","('union', 'pearson')","('union', 'spearman')","('union', 'number_variables')","('intersection', 'pearson')","('intersection', 'spearman')","('intersection', 'number_variables')"
0,Abdomen,Abdomen,Alcohol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abdomen,Abdomen,AnthropometryBodySize,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abdomen,Abdomen,AnthropometryImpedance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Abdomen,Abdomen,ArterialStiffness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abdomen,Abdomen,BloodBiochemistry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
100183,set_instances23,set_instances23,All_Biomarkers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100184,set_instances23,set_instances23,All_Phenotypes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100185,set_instances23,set_instances23,All_Diseases,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100186,set_instances23,set_instances23,All_Environmental,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
correlations = d.set_index(["dimension_1", "dimension_2", "category"])
correlations.columns = pd.MultiIndex.from_tuples(
    list(map(eval, correlations.columns.tolist())), names=["category", "variable"]
)
correlations

NameError: name 'd' is not defined

In [46]:
sorted(phenotypic.index.get_level_values(0).drop_duplicates())

['Abdomen',
 'AbdomenLiver',
 'AbdomenPancreas',
 'Arterial',
 'ArterialCarotids',
 'ArterialPulseWaveAnalysis',
 'Biochemistry',
 'BiochemistryBlood',
 'BiochemistryUrine',
 'Brain',
 'BrainCognitive',
 'BrainMRI',
 'Eyes',
 'EyesAll',
 'EyesFundus',
 'EyesOCT',
 'Hearing',
 'Heart',
 'HeartECG',
 'HeartMRI',
 'ImmuneSystem',
 'Lungs',
 'Musculoskeletal',
 'MusculoskeletalFullBody',
 'MusculoskeletalHips',
 'MusculoskeletalKnees',
 'MusculoskeletalScalars',
 'MusculoskeletalSpine',
 'PhysicalActivity',
 'set',
 'set_instances01',
 'set_instances1.5x',
 'set_instances23']